In [1]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import word_tokenize

import nltk
from nltk.corpus import stopwords
from nltk import tokenize
from nltk.util import ngrams

from collections import Counter    


from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.probability import FreqDist
pd.set_option('display.max_colwidth', -1)

<ipython-input-1-69d0281299de>:17: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
#Stopwords
stop_words=[]
stop_words = (pd.read_excel('C:/Users/Usuario/Desktop/Respositorio archivos .py/NLP/Datasets/stop_words_sin_tilde1.xlsx')) 
stop_words=list(stop_words['stopwords'])

# new_stopwords=['dia','no','si','hábiles','15','días','respuesta','correo','electrónico','autoriza','adjunta','pantallazo','indica','tratamiento','datos','personales'
#               'tiempo', 'gestión','1','usuario','comunica','confronta','exitoso','usuario','solicita','autorización',
#               'ncc','nse','ncorreo','ncedula','ncedula','nnombre','noperación']
# for i in range(0,len(new_stopwords)):
#     stop_words.append(new_stopwords[i])
    
    
eliminar_frases=['estas en valledupar y necesitas un efectyoficial visitanos cll','barrio san joaquin horario extendido ubalvilla',
                'imagen enviada desde el cliente','httapipuresocialcxgetfileddccfaee',
                'santa barbara','buenos dias','podrian', 'por favor', 'punto de hacienda',
                 'de apertura de punto','buenas tardes','buenas tarde','muchas gracias']

#Frases que no aportan valor y no son tenídas en cuenta
# stop_phrases=[' _ _ ',' _ _ ''nombre_del_proyecto','descripción_del_asesor','número_de_caso','realizar_un_pago',
# 'día_de_hoy','usuaria_informa_que','DÍA_DE_HOY','USUARIA_INFORMA_QUE','ASESORA_ELIZABETH_HERNÁNDEZ',
# 'RADICA_ASESORA_ELIZABETH','OPERACIÓN_NOMBRE_DEL','DÍA_DE_AYER','DÍA_DE_HOY']

In [3]:
#IMPORTAR ARCHIVO JSON
import json
import pandas as pd
import re

f = open('C:/Users/Usuario/Downloads/comentarios genesys.json', encoding="utf8")
p=json.load(f)
df = pd.DataFrame.from_dict(p['conversaciones'])
# df

In [4]:
#Tomar el primer msj

df['msj_cliente']=df['MENSAJE'].apply(lambda x: re.findall('Cliente:(.*)',(x)))
# df['msj_cliente']=df['msj_cliente'].apply(lambda x: x[0])
df['msj_cliente']=df['msj_cliente'].apply(lambda x: re.sub('escríbenos al Whatsapp.*','',str(x)))


In [5]:
df.rename(columns={'msj_cliente':'Descripción'},inplace=True)
Encuestas=df

In [6]:
Encuestas['Descripción']=Encuestas['Descripción'].astype(str)
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: x.lower())
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub('se\s[a-z]{1,10}','',str(x)))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub('á','a',str(x)))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub('é','e',str(x)))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub('í','i',str(x)))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub('ó','o',str(x)))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub('ú','u',str(x)))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub(r"""[\n\t]"""," ",x))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub("""[-*\_¡[!@#$:)(.;–,¿?&'_>/]{1,4}""","",x))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub("""[]]{1,4}""","",x))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub(r"""[0-9]""","",x))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub(r"""https.*?""","",x))


emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
# Encuestas['emojis']=Encuestas['Descripción'].apply(lambda x: emoji_pattern.findall(r'',str(x)))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: emoji_pattern.sub(r'', x))


In [7]:
for f in eliminar_frases:
    Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub(f,'',str(x)))

In [8]:
Encuestas=Encuestas[Encuestas['Descripción']!='']
Encuestas.reset_index(inplace=True)

In [9]:
Encuestas.drop_duplicates(subset=['MENSAJE'],inplace=True)

<ipython-input-9-1049b7eb1c9a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Encuestas.drop_duplicates(subset=['MENSAJE'],inplace=True)


In [11]:
Encuestas.head(30)

,index,CONVERSATIONID,PARTICIPANTID,PARTICIPANTNAME,PURPOSE,WRAPUPCODE,MENSAJE,WRAPUPNAME,WRAPUPNOTE,SESSIONID,NAME,division.name,TIO_MENSAJE,FECHA_MENSAJE,PROCESO_ID,FECHA_PROCESO,Descripción
0,0,263e8816-1152-414c-939a-08dcd86821b3,a1a89825-af5a-439b-a3b3-ef968a7a1113,None,agent,435db743-0875-49bd-9add-f2b5c8118320,📣 20/10/2021 07:55:50 **efecty:** [🌐 URL REVIEW](https://facebook.com/10224885362577536) 👍\n\n📣 20/10/2021 07:55:50 **Cliente:** Excelente empresa. agilidad y seguridad en sus servicios #Yoelijoaefecty #efectyGirandoatuLado #Efecty\n\n⚠️ -> 📣,5-Elogios Marca,None,2301590a-6cc4-42f7-a7d8-1bd2711752bf,Facebook Muro,Cliente Natural,standard,2021-10-20T12:55:51.325Z,228,21-10-2021 23:17:47,excelente empresa agilidad y seguridad en sus servicios yoelijoaefecty efectygirandoatulado efecty
1,2,96048c22-744b-4e64-b725-238cd8c190d4,f84ba4a3-bd49-43d4-9bfa-35ac04e5dd88,Facebook Muro,acd,None,"📣 24/07/2021 15:20:34 **Cliente:** escríbenos al Whatsapp 317 319 5672 o regalanos el tuyo para escribirte de una (no tenemos messenger) Podrás prestar los servicios Efecty sin requisitos\n\n📣 24/07/2021 15:20:34 **efecty:** [🌐 URL POST](https://www.facebook.com/165852228106/posts/10158364466443107/)\n\n📣 29/07/2021 08:27:36 **Cliente:** Jeferson Ortiz claro. escríbanos al whatsapp 317 319 5672 o déjanos el tuyo para escribirte\n\n📣 29/07/2021 08:27:36 **efecty:** [🌐 URL POST](https://www.facebook.com/165852228106/posts/10158364466443107/)\n\n📣 29/07/2021 20:03:16 **efecty:** [🌐 URL POST](https://www.facebook.com/165852228106/posts/10158364466443107/)\n\n📣 29/07/2021 20:03:16 **Cliente:** Escríbenos. te enseñamos a hacer giros nacionales ganando el 100% de la Comisión y sin ningún requisito ni papeleo... WHATSAPP 317 319 5672 con Efecty solo ganarás el 25% (Y)\n\n📣 29/07/2021 20:06:32 **Cliente:** y sí. muy atrasados. hoy es posible tener todo lo que ellos ofrecen sin papeleo ni requisitos\n\n📣 21/08/2021 23:28:33 **efecty:** [🌐 URL POST](https://www.facebook.com/165852228106/posts/10158364466443107/)\n\n📣 21/08/2021 23:28:33 **Cliente:** hola. no manejamos messenger (que casualmente estaba mirando face). te envío la información. pero para contacto futuro puedes escribirme directo al whatsapp wa.me/573173195672 (solo haz clic aquí)\n\n📣 21/08/2021 23:30:35 **Cliente:** ya te envié la información\n\n📣 21/08/2021 23:34:11 **efecty:** [🌐 URL POST](https://www.facebook.com/165852228106/posts/10158484216453107/)\n\n📣 21/08/2021 23:34:11 **Cliente:** Duke DK todo se puede pagar en un solo lugar y sin salir de casa. recibimos todas las formas de pago del país: Nequi. Movii. Daviplata y todos los bancos. todos los corresponsales bancarios e incluso Efecty sin comisión.\n\nWhatsapp 317 319 5672\n\n📣 21/08/2021 23:35:24 **Cliente:** Yesica Vera Hola te prestamos servicios de Efecty y de multipagos (con todos los convenios del país) para pagar a nivel nacional e internacional de manera virtual o presencial . como prefieras. Escríbenos Whatsapp 317 319 5672\n\n📣 21/08/2021 23:35:24 **efecty:** [🌐 URL POST](https://www.facebook.com/165852228106/posts/10158477929483107/)\n\n📣 21/08/2021 23:37:26 **efecty:** [🌐 URL POST](https://www.facebook.com/165852228106/posts/10158470496203107/)\n\n📣 21/08/2021 23:37:26 **Cliente:** se pueden enviar giros efecty a través de ustedes (sin necesidad de ir a Efecty a ponerlo y para que reciban el dinero en un punto Efecty)?\n\n📣 20/10/2021 18:52:40 **efecty:** [🌐 URL POST](https://www.facebook.com/165852228106/posts/10158525403928107/)\n\n📣 20/10/2021 18:52:40 **Cliente:** P Sandra Jembuel Pagalos a través de nosotros y sin necesidad de salir de casa whatsapp 317 319 5672. todos los servicios y convenios del país\n\n📣 20/10/2021 18:54:13 **efecty:** [🌐 URL POST](https://www.facebook.com/165852228106/posts/10158525403928107/)\n\n📣 20/10/2021 18:54:13 **Cliente:** REGALAREMOS UN 60% de descuento en el primer giro Efecty a las 3 primeras personas que nos escriban ""QUIERO MI GIRO"". Whatsapp 317 319 5672.\n\n⚠️ -> 📣",N

In [10]:
pa_token=re.sub("""[,']""",'',str(list(Encuestas['Descripción'])))
tokenized_word=word_tokenize(pa_token)

In [11]:
qty_most_common=10
qty_words_ngram=4
max_stopwords=2
df=Encuestas
global Top_words
global qty_most_common
Top_words=[]
Encuestas2=pd.DataFrame()
ongrams=ngrams(tokenized_word,qty_words_ngram)
ongrams=list(zip(ongrams))
ongrams=pd.DataFrame(ongrams)
ongrams['r']=0
for i in range(0,qty_words_ngram):
    ongrams['val'+str(i)]=ongrams[0].apply(lambda x: x[i])
    ongrams['w'+str(i)]=0
    index_on_val=ongrams[ongrams['val'+str(i)].isin(stop_words)].index
    ongrams.loc[index_on_val,'w'+str(i)]=1
    ongrams['r']=ongrams['r']+ongrams['w'+str(i)]

ongrams=ongrams[(ongrams['r']<=max_stopwords)&(ongrams['w'+str(qty_words_ngram-1)]==0)] #La ultima condicion es para que no termine con un stopword
ongrams_=[]
ongrams.reset_index(inplace=True)

for i in range(len(ongrams[0])):
    ongrams_.append(re.sub("""[¡!@)#($:.;,¿?&'_]""","",str(ongrams[0][i]))) #Agregar nan


fdist = FreqDist((ongrams_))

df['frases']=''
for i in range(0,qty_most_common):
    index_frases=df[(df['Descripción'].str.contains(str(fdist.most_common(qty_most_common)[i][0])))&(df['frases']=='')].index
    index_frases_duplicadas=df[(df['Descripción'].str.contains(str(fdist.most_common(qty_most_common)[i][0])))&(df['frases']!='')].index
#Si, veo que esta en el top, pero que no está en las frases df['Descripción'], es porque son 2 palabras que entre medio tenian una frase que 
# se eliminó y queda un espacio grande entre medio que te lo considera dentro del top, pero cuando lo busca no lo encuentra,
# porque no existe tal cual, sino con un espacio de por medio y una frase sin sentido.
    df.loc[index_frases,'frases']=fdist.most_common(qty_most_common)[i][0]
    Top_words.append(fdist.most_common(qty_most_common)[i][0])
    aux2=df.loc[index_frases_duplicadas,:]
    aux2.loc[:,'frases']=fdist.most_common(qty_most_common)[i][0]
    Encuestas2=pd.concat([aux2,Encuestas2])
#     Encuestas2.drop(columns=['level_0'])
#     Encuestas2.reset_index(inplace=True)
#     Encuestas2.drop(columns=['level_0'])
df=pd.concat([df,Encuestas2])
Encuestas_con_frases=df[df['frases']!='']
#     df['frases']=df['frases'].apply(lambda x: x.upper())
#     Top_words=fdist.most_common(qty_most_common)
df

,index,CONVERSATIONID,PARTICIPANTID,PARTICIPANTNAME,PURPOSE,WRAPUPCODE,MENSAJE,WRAPUPNAME,WRAPUPNOTE,SESSIONID,NAME,division.name,TIO_MENSAJE,FECHA_MENSAJE,PROCESO_ID,FECHA_PROCESO,Descripción,frases
0,0,263e8816-1152-414c-939a-08dcd86821b3,a1a89825-af5a-439b-a3b3-ef968a7a1113,None,agent,435db743-0875-49bd-9add-f2b5c8118320,📣 20/10/2021 07:55:50 **efecty:** [🌐 URL REVIEW](https://facebook.com/10224885362577536) 👍\n\n📣 20/10/2021 07:55:50 **Cliente:** Excelente empresa. agilidad y seguridad en sus servicios #Yoelijoaefecty #efectyGirandoatuLado #Efecty\n\n⚠️ -> 📣,5-Elogios Marca,None,2301590a-6cc4-42f7-a7d8-1bd2711752bf,Facebook Muro,Cliente Natural,standard,2021-10-20T12:55:51.325Z,228,21-10-2021 23:17:47,excelente empresa agilidad y seguridad en sus servicios yoelijoaefecty efectygirandoatulado efecty,
1,2,96048c22-744b-4e64-b725-238cd8c190d4,f84ba4a3-bd49-43d4-9bfa-35ac04e5dd88,Facebook Muro,acd,None,"📣 24/07/2021 15:20:34 **Cliente:** escríbenos al Whatsapp 317 319 5672 o regalanos el tuyo para escribirte de una (no tenemos messenger) Podrás prestar los servicios Efecty sin requisitos\n\n📣 24/07/2021 15:20:34 **efecty:** [🌐 URL POST](https://www.facebook.com/165852228106/posts/10158364466443107/)\n\n📣 29/07/2021 08:27:36 **Cliente:** Jeferson Ortiz claro. escríbanos al whatsapp 317 319 5672 o déjanos el tuyo para escribirte\n\n📣 29/07/2021 08:27:36 **efecty:** [🌐 URL POST](https://www.facebook.com/165852228106/posts/10158364466443107/)\n\n📣 29/07/2021 20:03:16 **efecty:** [🌐 URL POST](https://www.facebook.com/165852228106/posts/10158364466443107/)\n\n📣 29/07/2021 20:03:16 **Cliente:** Escríbenos. te enseñamos a hacer giros nacionales ganando el 100% de la Comisión y sin ningún requisito ni papeleo... WHATSAPP 317 319 5672 con Efecty solo ganarás el 25% (Y)\n\n📣 29/07/2021 20:06:32 **Cliente:** y sí. muy atrasados. hoy es posible tener todo lo que ellos ofrecen sin papeleo ni requisitos\n\n📣 21/08/2021 23:28:33 **efecty:** [🌐 URL POST](https://www.facebook.com/165852228106/posts/10158364466443107/)\n\n📣 21/08/2021 23:28:33 **Cliente:** hola. no manejamos messenger (que casualmente estaba mirando face). te envío la información. pero para contacto futuro puedes escribirme directo al whatsapp wa.me/573173195672 (solo haz clic aquí)\n\n📣 21/08/2021 23:30:35 **Cliente:** ya te envié la información\n\n📣 21/08/2021 23:34:11 **efecty:** [🌐 URL POST](https://www.facebook.com/165852228106/posts/10158484216453107/)\n\n📣 21/08/2021 23:34:11 **Cliente:** Duke DK todo se puede pagar en un solo lugar y sin salir de casa. recibimos todas las formas de pago del país: Nequi. Movii. Daviplata y todos los bancos. todos los corresponsales bancarios e incluso Efecty sin comisión.\n\nWhatsapp 317 319 5672\n\n📣 21/08/2021 23:35:24 **Cliente:** Yesica Vera Hola te prestamos servicios de Efecty y de multipagos (con todos los convenios del país) para pagar a nivel nacional e internacional de manera virtual o presencial . como prefieras. Escríbenos Whatsapp 317 319 5672\n\n📣 21/08/2021 23:35:24 **efecty:** [🌐 URL POST](https://www.facebook.com/165852228106/posts/10158477929483107/)\n\n📣 21/08/2021 23:37:26 **efecty:** [🌐 URL POST](https://www.facebook.com/165852228106/posts/10158470496203107/)\n\n📣 21/08/2021 23:37:26 **Cliente:** se pueden enviar giros efecty a través de ustedes (sin necesidad de ir a Efecty a ponerlo y para que reciban el dinero en un punto Efecty)?\n\n📣 20/10/2021 18:52:40 **efecty:** [🌐 URL POST](https://www.facebook.com/165852228106/posts/10158525403928107/)\n\n📣 20/10/2021 18:52:40 **Cliente:** P Sandra Jembuel Pagalos a través de nosotros y sin necesidad de salir de casa whatsapp 317 319 5672. todos los servicios y convenios del país\n\n📣 20/10/2021 18:54:13 **efecty:** [🌐 URL POST](https://www.facebook.com/165852228106/posts/10158525403928107/)\n\n📣 20/10/2021 18:54:13 **Cliente:** REGALAREMOS UN 60% de descuento en el primer giro Efecty a las 3 primeras personas que nos escriban ""QUIERO MI GIRO"". Whatsapp 317 319 5672.\n\n⚠️

In [12]:
qty_most_common=10
def gen_frases(qty_words_ngram,max_stopwords,df):
    global Top_words
    global qty_most_common
    Top_words=[]
    Encuestas2=pd.DataFrame()
    ongrams=ngrams(tokenized_word,qty_words_ngram)
    ongrams=list(zip(ongrams))
    ongrams=pd.DataFrame(ongrams)
    ongrams['r']=0
    for i in range(0,qty_words_ngram):
        ongrams['val'+str(i)]=ongrams[0].apply(lambda x: x[i])
        ongrams['w'+str(i)]=0
        index_on_val=ongrams[ongrams['val'+str(i)].isin(stop_words)].index
        ongrams.loc[index_on_val,'w'+str(i)]=1
        ongrams['r']=ongrams['r']+ongrams['w'+str(i)]

    ongrams=ongrams[(ongrams['r']<=max_stopwords)&(ongrams['w'+str(qty_words_ngram-1)]==0)] #La ultima condicion es para que no termine con un stopword
    ongrams_=[]
    ongrams.reset_index(inplace=True)

    for i in range(len(ongrams[0])):
        ongrams_.append(re.sub("""[¡!@)#($:.;,¿?&'_]""","",str(ongrams[0][i]))) #Agregar nan

    
    fdist = FreqDist((ongrams_))
    
    df['frases']=''
    for i in range(0,qty_most_common):
        index_frases=df[(df['Descripción'].str.contains(str(fdist.most_common(qty_most_common)[i][0])))&(df['frases']=='')].index
        index_frases_duplicadas=df[(df['Descripción'].str.contains(str(fdist.most_common(qty_most_common)[i][0])))&(df['frases']!='')].index
#Si, veo que esta en el top, pero que no está en las frases df['Descripción'], es porque son 2 palabras que entre medio tenian una frase que 
# se eliminó y queda un espacio grande entre medio que te lo considera dentro del top, pero cuando lo busca no lo encuentra,
# porque no existe tal cual, sino con un espacio de por medio y una frase sin sentido.
        df.loc[index_frases,'frases']=fdist.most_common(qty_most_common)[i][0]
        Top_words.append(fdist.most_common(qty_most_common)[i][0])
        aux2=df.loc[index_frases_duplicadas,:]
        aux2.loc[:,'frases']=fdist.most_common(qty_most_common)[i][0]
        Encuestas2=pd.concat([aux2,Encuestas2])
#     Encuestas2.drop(columns=['level_0'])
#     Encuestas2.reset_index(inplace=True)
#     Encuestas2.drop(columns=['level_0'])
    df=pd.concat([df,Encuestas2])
    Encuestas_con_frases=df[df['frases']!='']
#     df['frases']=df['frases'].apply(lambda x: x.upper())
#     Top_words=fdist.most_common(qty_most_common)
    return df


In [13]:
# R_frases1=gen_frases(1,0,20,Encuestas)
# R_frases1=R_frases1[R_frases1['frases']!='']
# top_1_palabra=pd.DataFrame(Top_words)

R_frases2=gen_frases(2,0,Encuestas) 
R_frases2=R_frases2[R_frases2['frases']!='']
top_2_palabras=pd.DataFrame(Top_words)

R_frases3=gen_frases(3,1,Encuestas) 
R_frases3=R_frases3[R_frases3['frases']!='']
top_3_palabras=pd.DataFrame(Top_words)

R_frases4=gen_frases(4,2,Encuestas) 
R_frases4=R_frases4[R_frases4['frases']!='']
top_4_palabras=pd.DataFrame(Top_words)


In [14]:
Encuestas3=pd.concat([Encuestas,R_frases2])
# Encuestas3=pd.concat([Encuestas3,R_frases2])
Encuestas3=pd.concat([Encuestas3,R_frases3])
Encuestas3=pd.concat([Encuestas3,R_frases4])
Encuestas3.sort_values(by='index',inplace=True)
# Encuestas3.drop(columns=['level_0','index'],inplace=True)
Encuestas3.reset_index(inplace=True)

In [15]:
#PROCESO PARA LIMIPAR FRASES SEMEJANTES DENTRO DE 4 PALABRAS. Repetir este proceso para 3 y 2 palabras

eliminar=[]
index_eliminar_2=[]
index_eliminar_3=[]
index_eliminar_4=[]
top_2_palabras_n=top_2_palabras
top_4_palabras_n=top_4_palabras
top_3_palabras_n=top_3_palabras

for j in range(0,qty_most_common): 
    
    #Pensado para 20 palabras mas comunes de 4 palabras. Compara contra las de 4 y las de 3
    qs=re.findall('[\w]{1,10}',top_4_palabras_n[0][j])
    # 4 palabras solo va a ir contra 3
    for i in range(0,len(qs)):
        for p1 in range(0,qty_most_common):
            try: #Mira si hay 3 palabras segúidas repetidas dentro de los 4grams, si es asi lo elimina
                if(((qs[i]+' '+qs[i+1]+' '+qs[i+2]) in top_4_palabras_n[0][p1])&(p1!=j)):
                    eliminar.append(top_4_palabras_n[0][p1])
                    top_4_palabras_n[0][p1]='-4'
                    index_eliminar_4.append(p1)
            except:
                pass

            
            try: #Mira si hay 2 palabras segúidas repetidas dentro de los 3grams, si es asi lo elimina
                if((qs[i]+' '+qs[i+1]) in top_3_palabras_n[0][p1]):
                    eliminar.append(top_3_palabras_n[0][p1])
                    index_eliminar_3.append(p1)
                    top_3_palabras_n[0][p1]='-4'
            except:
                pass

            try: #Mira si hay 2 palabras segúidas repetidas dentro de los 2grams, si es asi lo elimina
                if((qs[i]+' '+qs[i+1]) in top_2_palabras_n[0][p1]):
                    eliminar.append(top_2_palabras_n[0][p1])
                    index_eliminar_2.append(p1)
                    top_2_palabras_n[0][p1]='-4'
            except:
                pass
            
    #Pensado para 20 palabras mas comunes de 3 palabras, compara contra las de 3 y las de 2
    qs3=re.findall('[\w]{1,10}',top_3_palabras_n[0][j])
    # 3 palabras solo va a ir contra 2
    for i in range(0,len(qs3)):
        for p1 in range(0,qty_most_common):
            try: #Mira si hay 3 palabras segúidas repetidas dentro de los fourgrams, si es asi lo elimina
                if(((qs3[i]+' '+qs3[i+1]) in top_3_palabras_n[0][p1])&(p1!=j)):
                    eliminar.append(top_3_palabras_n[0][p1])
                    index_eliminar_3.append(p1)
                    top_3_palabras_n[0][p1]='-3'
            except:
                pass

            try: #Mira si hay 2 palabras segúidas repetidas dentro de los fourgrams, si es asi lo elimina
                if((qs3[i]+' '+qs3[i+1]) in top_2_palabras_n[0][p1]):
                    eliminar.append(top_2_palabras_n[0][p1])
                    index_eliminar_2.append(p1)
                    top_2_palabras_n[0][p1]='-3'
            except:
                pass
            

#ELIMINA LAS FRASES QUE EL PROCESO ELIGIÓ LIMPIAR
for b in range(0,len(eliminar)):
    index_a_eliminar=Encuestas3[Encuestas3['frases']==eliminar[b]].index
    Encuestas3.loc[index_a_eliminar,'frases']='' 


In [16]:
Encuestas3['frases'].unique()

array(['', 'interes personal les vale', 'puta empresa que solo',
       'robado y estos hps', 'estos hps lo unico',
       'mierda las calamidades', 'piensa en el interes',
       'señora madre dueña', 'efecty una puta empresa', 'solo piensa',
       'vale mierda', 'ya para poder abrir', 'enviar giros',
       'recibimos todas', 'hacer giros', 'sin necesidad'], dtype=object)

In [17]:
ruta3= "C:/Users/Usuario/Desktop/Respositorio archivos .py/borrador/Frases_redes4.xlsx"

In [18]:
from pandas import ExcelWriter
from openpyxl import load_workbook

#Ingreso las bases a hojas al archivo
writer = pd.ExcelWriter(ruta3, engine='openpyxl')
Encuestas.to_excel(writer, index=True)
# Encuestas3[['MENSAJE','Descripción','frases']].to_excel(writer, index=True)
writer.save()
writer.close()

In [19]:
#DESDE ACAAA

In [20]:
# https://www.kaggle.com/haibaral/spanish-word2vec/notebook
import gensim
import gensim.downloader as api
from gensim.models import KeyedVectors

In [21]:
model = gensim.models.KeyedVectors.load_word2vec_format('C:/Users/Usuario/Downloads/archive/SBW-vectors-300-min5.txt')

KeyboardInterrupt: 

In [ ]:
# 'ENVIAR TIRILLA', 'INDICA ENVIAR', 'NUMERO DE CEDULA',
#        'RESPUESTA LA EMITE EL', 'LA EMITE EL CE',
#        'PROYECTO ERRADO PENSABA PAGAR', 'A UN PROYECTO ERRADO',
#        'DEVOLUCION DE OS ERRADA', 'CLARO FIJO', 'PROYECTO CLARO',
#        'CORREO SAC', 'POR ERROR GENERARON', 'POR ERROR GENERARON EL',
#        'ERROR GENERARON EL PAGO', 'INFORMA QUE POR ERROR',
#        'QUE POR ERROR GENERARON', 'CON REFERENCIA ERRADA',
#        'PAGO CON REFERENCIA ERRADA', 'ERROR GENERARON',
#        'ORDEN DE SERVICIO', 'NUMERO DE REFERENCIA',
#        'NORMATIVIDAD ARTICULO', 'TIEMPO DE RESPUESTA',
#        'TIEMPO DE RESPUESTA DE', 'RELACION DE GIROS', 'CLARO MOVIL',
#        'HASTA LA FECHA', 'NOVAVENTA INTEGRACION', 'PUNTO DE ATENCION',
#        'DE TRADICION Y LIBERTAD', 'CON NUMERO DE REFERENCIA',
#        'OS ERRADA AL PROYECTO', 'ERRADA AUTORIZA',
#        'LA DEVOLUCION DEL DINERO', 'CEDULA DE CIUDADANIA',
#        'GIROS ENVIADOS', 'CE ENVIO', 'TRATAMIENTO DE DATOS',
#        'ATENCION PRESTADA', 'XD XD', 'P M', 'AUTORIZA TRATAMIENTO DE',
#        'AUTORIZA TRATAMIENTO', 'CONFRONTA EXITOSO', 'GESTION AUTORIZA',
#        'LA EXPIDE EL CE', 'BAJO LA OS INDICANDO',
#        'TIEMPO DE GESTION RESPUESTA'

In [ ]:
model.most_similar('ágil')
# model.words_closer_than('belleza','mujer') #Palabras cercanas a w1 y lejanas a w2
# model.doesnt_match(['deporte','futbol','socer','bota'])
model.n_similarity(['tiempo'],['ellos','no','quieren','porque','hay','mucha','espera','en','trámite'])

# model.most_similar_cosmul(positive='belleza',negative='mujer'). Raro


In [ ]:
pip install wikipediaapi

In [ ]:
import nltk
nltk.corpus.gutenberg.words('austen-emma.txt')

In [ ]:
# https://relopezbriega.github.io/blog/2017/09/23/procesamiento-del-lenguaje-natural-con-python/
import wikipedia 
wp = wikipedia( lang='es', version='latest')
wp.download()